# DeepStar Convolutional Neural Network

In [1]:
import os, sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

##os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import time, onnx, loader, torch, torchvision
import torch.utils.data as utils
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

from DeepStar.V3.Model import DeepStar, PathDataLoader

tensor(1.0000)


In [2]:
load = False
save = True
save_name = "Trained Models/deep_star_v2.pt"

num_workers = 16
seed = 0
use_gpu = True
small_batch = False

image_size = (256, 256)
prediction_size = (32, 32)

data_folder = "../V2/data"
batch_size = 10
n_epochs = 20000
shuffle_data = True

lr = 0.001
momentum = 0.9

In [3]:
net = DeepStar(prediction_size)

if (load):
    net.load_state_dict(torch.load(save_name))
    
if (use_gpu):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net.to(device)

## Load data

Here we load both the training data and validation data into different loaders. There are methods for automatically separating out training and validation data from a set but we do it manually.

In [4]:
train_data = PathDataLoader(f'{data_folder}/images/', prediction_size)
train_loader = utils.DataLoader(train_data, batch_size=batch_size, shuffle=shuffle_data, num_workers=num_workers)

val_data = PathDataLoader(f'{data_folder}/validation/', prediction_size)
val_loader = utils.DataLoader(val_data, batch_size=batch_size, shuffle=shuffle_data, num_workers=num_workers)

## Configure output for Tensor Board

Here we write to special files that enable Tensor Board to visualize it. To open Tensor Board select the data/images/ folder and click the Tensorboard button.

In [5]:
#writer = SummaryWriter(f'{data_folder}/images')

#dataiter = iter(train_loader)
#images, labels = dataiter.next()

#img_grid = torchvision.utils.make_grid(images)
#writer.add_image('dataset', img_grid)

#writer.add_graph(net, images)
#writer.close()

## Train The Model

In [ ]:
n_batches = len(train_loader)

loss = net.get_loss()
optimizer = net.get_optim(lr, momentum)
training_start_time = time.time()

print(f'Training with {n_batches} training batches and {len(val_loader)} validation batches.')
for epoch in range(n_epochs):

    running_loss = 0.0
    print_every = n_batches // 10
    start_time = time.time()
    total_train_loss = 0

    for i, data in enumerate(train_loader, 0):
        img, points, labels = data

        labels = labels.squeeze_()
        if (use_gpu):
            img, points, labels = img.to(device), points.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        #print(labels)

        output = net(img, points)
        loss_size = loss(output, labels)
        loss_size.backward()
        optimizer.step()
            
        
        running_loss += loss_size.data.item()
        total_train_loss += loss_size.data.item()

        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss: {:.3f} took: {:.2f}s".format(epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))

            running_loss = 0.0
            start_time = time.time()
            
        total_val_loss = 0
    for img, points, labels in val_loader:
        
        labels = labels.squeeze_()
        if (use_gpu):
            img, points, labels = img.to(device), points.to(device), labels.to(device)

        val_outputs = net(img, points)
        val_loss_size = loss(val_outputs, labels)
        total_val_loss += val_loss_size.data.item()

    print("Validation loss = {:.3f}".format(total_val_loss / len(val_loader)))
    
    if (save):
        print(f'saving network to \'{save_name}\'')
        torch.save(net.state_dict(), f'{save_name}')

print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

Training with 6138 training batches and 21 validation batches.
Epoch 1, 10% 	 train_loss: 4.417 took: 19.38s
Epoch 1, 20% 	 train_loss: 4.451 took: 18.93s
Epoch 1, 30% 	 train_loss: 4.450 took: 18.90s
Epoch 1, 40% 	 train_loss: 4.515 took: 18.90s
Epoch 1, 50% 	 train_loss: 4.454 took: 18.91s
Epoch 1, 60% 	 train_loss: 4.491 took: 18.90s
Epoch 1, 70% 	 train_loss: 4.528 took: 18.90s
Epoch 1, 80% 	 train_loss: 4.486 took: 18.91s
Epoch 1, 90% 	 train_loss: 4.516 took: 18.89s
Validation loss = 4.459
saving network to 'Trained Models/deep_star_v1'
Epoch 2, 10% 	 train_loss: 4.482 took: 19.41s
Epoch 2, 20% 	 train_loss: 4.522 took: 18.91s
Epoch 2, 30% 	 train_loss: 4.527 took: 18.92s
Epoch 2, 40% 	 train_loss: 4.555 took: 19.77s
Epoch 2, 50% 	 train_loss: 4.483 took: 19.66s
Epoch 2, 60% 	 train_loss: 4.554 took: 18.91s
Epoch 2, 70% 	 train_loss: 4.644 took: 18.91s
Epoch 2, 80% 	 train_loss: 4.617 took: 18.93s
Epoch 2, 90% 	 train_loss: 4.652 took: 18.90s
Validation loss = 4.387
saving networ

Epoch 17, 80% 	 train_loss: 4.628 took: 18.67s
Epoch 17, 90% 	 train_loss: 4.611 took: 18.67s
Validation loss = 4.408
saving network to 'Trained Models/deep_star_v1'
Epoch 18, 10% 	 train_loss: 4.564 took: 19.14s
Epoch 18, 20% 	 train_loss: 4.560 took: 18.64s
Epoch 18, 30% 	 train_loss: 4.525 took: 18.65s
Epoch 18, 40% 	 train_loss: 4.542 took: 18.65s
Epoch 18, 50% 	 train_loss: 4.613 took: 18.65s
Epoch 18, 60% 	 train_loss: 4.564 took: 18.66s
Epoch 18, 70% 	 train_loss: 4.597 took: 18.65s
Epoch 18, 80% 	 train_loss: 4.578 took: 18.66s
Epoch 18, 90% 	 train_loss: 4.542 took: 18.65s
Validation loss = 4.429
saving network to 'Trained Models/deep_star_v1'
Epoch 19, 10% 	 train_loss: 4.563 took: 19.14s
Epoch 19, 20% 	 train_loss: 4.646 took: 18.65s
Epoch 19, 30% 	 train_loss: 4.565 took: 18.65s
Epoch 19, 40% 	 train_loss: 4.561 took: 18.65s
Epoch 19, 50% 	 train_loss: 4.539 took: 18.66s
Epoch 19, 60% 	 train_loss: 4.532 took: 18.65s
Epoch 19, 70% 	 train_loss: 4.505 took: 18.65s
Epoch 19, 

Epoch 34, 40% 	 train_loss: 5.065 took: 18.64s
Epoch 34, 50% 	 train_loss: 5.009 took: 18.62s
Epoch 34, 60% 	 train_loss: 5.043 took: 18.63s
Epoch 34, 70% 	 train_loss: 4.992 took: 18.64s
Epoch 34, 80% 	 train_loss: 4.953 took: 18.64s
Epoch 34, 90% 	 train_loss: 4.967 took: 18.64s
Validation loss = 4.878
saving network to 'Trained Models/deep_star_v1'
Epoch 35, 10% 	 train_loss: 4.967 took: 19.15s
Epoch 35, 20% 	 train_loss: 4.949 took: 18.65s
Epoch 35, 30% 	 train_loss: 5.009 took: 18.64s
Epoch 35, 40% 	 train_loss: 4.966 took: 18.65s
Epoch 35, 50% 	 train_loss: 4.963 took: 18.64s
Epoch 35, 60% 	 train_loss: 5.000 took: 18.65s
Epoch 35, 70% 	 train_loss: 4.947 took: 18.66s
Epoch 35, 80% 	 train_loss: 4.955 took: 18.64s
Epoch 35, 90% 	 train_loss: 4.905 took: 18.65s
Validation loss = 5.124
saving network to 'Trained Models/deep_star_v1'
Epoch 36, 10% 	 train_loss: 4.986 took: 19.14s
Epoch 36, 20% 	 train_loss: 4.914 took: 18.65s
Epoch 36, 30% 	 train_loss: 4.931 took: 18.66s
Epoch 36, 

Validation loss = 4.952
saving network to 'Trained Models/deep_star_v1'
Epoch 51, 10% 	 train_loss: 5.080 took: 19.80s
Epoch 51, 20% 	 train_loss: 4.996 took: 19.43s
Epoch 51, 30% 	 train_loss: 5.106 took: 19.44s
Epoch 51, 40% 	 train_loss: 5.132 took: 19.45s
Epoch 51, 50% 	 train_loss: 5.003 took: 19.23s
Epoch 51, 60% 	 train_loss: 5.100 took: 18.90s
Epoch 51, 70% 	 train_loss: 5.092 took: 19.33s
Epoch 51, 80% 	 train_loss: 5.047 took: 19.48s
Epoch 51, 90% 	 train_loss: 4.989 took: 19.22s
Validation loss = 5.337
saving network to 'Trained Models/deep_star_v1'
Epoch 52, 10% 	 train_loss: 5.046 took: 21.20s
Epoch 52, 20% 	 train_loss: 5.030 took: 19.51s
Epoch 52, 30% 	 train_loss: 5.068 took: 19.42s
Epoch 52, 40% 	 train_loss: 5.153 took: 19.80s
Epoch 52, 50% 	 train_loss: 5.088 took: 19.46s
Epoch 52, 60% 	 train_loss: 5.267 took: 19.18s
Epoch 52, 70% 	 train_loss: 5.151 took: 19.20s
Epoch 52, 80% 	 train_loss: 5.103 took: 19.20s
Epoch 52, 90% 	 train_loss: 4.971 took: 19.31s
Validation

Epoch 67, 60% 	 train_loss: 4.993 took: 18.70s
Epoch 67, 70% 	 train_loss: 4.952 took: 18.70s
Epoch 67, 80% 	 train_loss: 4.949 took: 18.70s
Epoch 67, 90% 	 train_loss: 4.922 took: 18.70s
Validation loss = 4.915
saving network to 'Trained Models/deep_star_v1'
Epoch 68, 10% 	 train_loss: 4.973 took: 19.15s
Epoch 68, 20% 	 train_loss: 4.959 took: 18.67s
Epoch 68, 30% 	 train_loss: 4.939 took: 18.67s
Epoch 68, 40% 	 train_loss: 4.963 took: 18.68s
Epoch 68, 50% 	 train_loss: 4.969 took: 18.67s
Epoch 68, 60% 	 train_loss: 4.946 took: 18.68s
Epoch 68, 70% 	 train_loss: 4.911 took: 18.67s
Epoch 68, 80% 	 train_loss: 4.983 took: 18.67s
Epoch 68, 90% 	 train_loss: 5.043 took: 18.67s
Validation loss = 4.833
saving network to 'Trained Models/deep_star_v1'
Epoch 69, 10% 	 train_loss: 4.971 took: 19.16s
Epoch 69, 20% 	 train_loss: 5.016 took: 18.69s
Epoch 69, 30% 	 train_loss: 4.927 took: 18.69s
Epoch 69, 40% 	 train_loss: 4.941 took: 18.69s
Epoch 69, 50% 	 train_loss: 4.969 took: 18.69s
Epoch 69, 

Epoch 84, 20% 	 train_loss: 4.968 took: 18.67s
Epoch 84, 30% 	 train_loss: 5.036 took: 18.68s
Epoch 84, 40% 	 train_loss: 5.052 took: 18.68s
Epoch 84, 50% 	 train_loss: 4.989 took: 18.70s
Epoch 84, 60% 	 train_loss: 5.049 took: 18.68s
Epoch 84, 70% 	 train_loss: 5.042 took: 18.69s
Epoch 84, 80% 	 train_loss: 4.992 took: 18.68s
Epoch 84, 90% 	 train_loss: 5.037 took: 18.68s
Validation loss = 5.156
saving network to 'Trained Models/deep_star_v1'
Epoch 85, 10% 	 train_loss: 5.058 took: 19.18s
Epoch 85, 20% 	 train_loss: 5.117 took: 18.69s
Epoch 85, 30% 	 train_loss: 5.023 took: 18.69s
Epoch 85, 40% 	 train_loss: 5.061 took: 18.70s
Epoch 85, 50% 	 train_loss: 5.059 took: 18.69s
Epoch 85, 60% 	 train_loss: 5.007 took: 18.69s
Epoch 85, 70% 	 train_loss: 5.044 took: 18.69s
Epoch 85, 80% 	 train_loss: 5.029 took: 18.70s
Epoch 85, 90% 	 train_loss: 5.048 took: 18.70s
Validation loss = 5.046
saving network to 'Trained Models/deep_star_v1'
Epoch 86, 10% 	 train_loss: 4.997 took: 19.16s
Epoch 86, 

Epoch 100, 80% 	 train_loss: 5.576 took: 18.70s
Epoch 100, 90% 	 train_loss: 5.586 took: 18.70s
Validation loss = 5.764
saving network to 'Trained Models/deep_star_v1'
Epoch 101, 10% 	 train_loss: 5.613 took: 19.16s
Epoch 101, 20% 	 train_loss: 5.570 took: 18.67s
Epoch 101, 30% 	 train_loss: 5.613 took: 18.69s
Epoch 101, 40% 	 train_loss: 5.587 took: 18.68s
Epoch 101, 50% 	 train_loss: 5.555 took: 18.68s
Epoch 101, 60% 	 train_loss: 5.568 took: 18.70s
Epoch 101, 70% 	 train_loss: 5.550 took: 18.68s
Epoch 101, 80% 	 train_loss: 5.601 took: 18.68s
Epoch 101, 90% 	 train_loss: 5.583 took: 18.68s
Validation loss = 5.634
saving network to 'Trained Models/deep_star_v1'
Epoch 102, 10% 	 train_loss: 5.569 took: 19.17s
Epoch 102, 20% 	 train_loss: 5.588 took: 18.69s
Epoch 102, 30% 	 train_loss: 5.578 took: 18.69s
Epoch 102, 40% 	 train_loss: 5.566 took: 18.68s
Epoch 102, 50% 	 train_loss: 5.611 took: 18.69s
Epoch 102, 60% 	 train_loss: 5.542 took: 18.68s
Epoch 102, 70% 	 train_loss: 5.620 took: